# Example: Recursive Implementation of Fibonacci Sequence Calculation
Recursion is a programming technique in which a function calls itself with a modified version of its input. This allows the function to repeat a process on a smaller scale, and the results of these smaller-scale processes can be combined to solve the original problem. 

We illustrate recursion concepts by looking at different implementations of the `fibonacci` sequence computation in [the `Compute.jl` file](src/Compute.jl). 

* First, let's use the [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) to compute the median time required to calculate the sequence $F_{0},\dots,F_{n}$ using the vanilla for-loop implementation of the `fibonacci` function that we explored previously. 
* Next, we'll benchmark a recursive implementation of the `fibonacci` function.
    * The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that computes sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument, which is updated in place.
* Lastly, we'll benchmark a recursive `fibonacci` function that uses memoization.
    * The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that uses memoization to speed up the computation of the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument.

## Setup
This example uses functions encoded in the `src` directory and external third-party packages. In [the `Include.jl` file](src/Include.jl), we load these functions to access them, set some required paths for this example and load external packages.

In [3]:
include("Include.jl");

Let's set the number of elements of the Fibonacci sequence $F_{0},\dots,F_{n}$ that we want to compute in the `n` variable:

In [5]:
n = 25; # compute the Fibonacci sequence from F0 to F25

## Case 1: Test the basal `for` loop implementation of `fibonacci` sequence
Let's use the [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) to compute the average time required to calculate the sequence $F_{0},\dots,F_{n}$ using the `vanilla` implementation of the `fibonacci` function (`for` loop based implementation that we explored previously). 

* The [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) exports the [@benchmarkable macro](https://juliaci.github.io/BenchmarkTools.jl/stable/reference/#BenchmarkTools.@benchmarkable-Tuple), which computes a function's runtime and memory profile. It runs the function many times and returns statistical information about its performance.
* _What_? Let's check out [the BenchmarkTools.jl documentation](https://github.com/JuliaCI/BenchmarkTools.jl) to see what some of this stuff, e.g., `samples` versus `evaluations`  or the `tune!` method, is.

In [7]:
result_basal = let
    test_run_basal = @benchmarkable fibonacci_for_loop_dict($(n));
    tune!(test_run_basal)
    result_basal = run(test_run_basal)
end

BenchmarkTools.Trial: 10000 samples with 134 evaluations.
 Range (min … max):  728.545 ns … 690.267 μs  ┊ GC (min … max):  0.00% … 99.84%
 Time  (median):     746.896 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   848.628 ns ±   6.900 μs  ┊ GC (mean ± σ):  10.14% ±  5.51%

   ▂▅▇█▇▆▄▃▃▄▃▃▃▂▂▁   ▁▁▂▂▂▂▁▂▁▁▁                               ▂
  ▇█████████████████████████████████▇▇▇▇▆▇▇▆▆▇▆▆▆▆▇▆▅▅▅▆▄▅▄▄▂▄▄ █
  729 ns        Histogram: log(frequency) by time        940 ns <

 Memory estimate: 1.78 KiB, allocs estimate: 7.

In [8]:
result_basal.times;

## Case 2: Test the `recursive` implementation of the `fibonacci` sequence
Let's benchmark a recursive implementation of the `fibonacci` function. The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that computes sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument. 
* __Q__: Recursion is cool. How does it perform relative to the `baseline` implementation of the `fibonacci` sequence calculation?

In [10]:
result_recursive = let
    result_dictionary = Dict{Int,Int}()
    test_run_recursive = @benchmarkable fibonacci!($(n), $(result_dictionary))
    tune!(test_run_recursive)
    result_recursive = run(test_run_recursive)
end

BenchmarkTools.Trial: 6696 samples with 1 evaluation.
 Range (min … max):  740.875 μs … 871.167 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     743.834 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   745.967 μs ±  10.782 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▁▇▂▁▁▄ ▁▂ ▁   ▁        ▁                                    ▁
  ███████████████████████████▇█▇▇▇▅▇▅▇▆▅▆▅▄▅▅▅▅▅▃▅▁▅▅▄▅▄▄▅▅▅▅▅▅ █
  741 μs        Histogram: log(frequency) by time        796 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
result_recursive.times;

## Case 3: Test the `recursive` implementation of `fibonacci` sequence with memoization
Finally, benchmark a recursive function that uses memoization. The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that uses memoization to speed up the computation of the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument.
* __Q__: Does the inclusion of the memoization change the runtime (or memory allocation) profile of the recursive implementation of the `fibonacci` sequence

In [13]:
result_recursive_memo = let
    result_dictionary_memo = Dict{Int,Int}()
    test_run_recursive_memo = @benchmarkable memoization_fibonacci!($(n), $(result_dictionary_memo))
    tune!(test_run_recursive_memo)
    result_recursive_memo = run(test_run_recursive_memo)
end

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  7.967 ns … 25.192 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.091 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.105 ns ±  0.403 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▂       ▇       █        ▄       ▁       ▃         ▁
  ▄▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█ █
  7.97 ns      Histogram: log(frequency) by time     8.26 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
result_recursive_memo.times;